### 라이브러리 불러오기

### 환경
* python 3.7
* scikit-learn : 0.22

### 참고
* scikit-learn 0.19 버전에서 일부 실행이 안되는 부분이 있음.

In [5]:
!pip install pandas

  Using cached pytz-2019.3-py2.py3-none-any.whl (509 kB)


In [2]:
import sklearn
print(sklearn.__version__)

0.22


In [6]:
import numpy as np
import pandas as pd

In [7]:
print(np.__version__)
print(pd.__version__)

1.18.1
1.0.3


In [8]:
# -----------------------------------
# 학습 데이터, 테스트 데이터 읽기
# -----------------------------------
# 학습 데이터, 테스트 데이터 읽기
train = pd.read_csv('../input/ch01-titanic/train.csv')
test = pd.read_csv('../input/ch01-titanic/test.csv')

# 학습 데이터을 종속 변수와 목적 변수로 나누기
train_x = train.drop(['Survived'], axis=1)
train_y = train['Survived']

# 테스트 데이터와 종속변수만 있기 때문에, 그대로 괜찮음.
test_x = test.copy()

In [9]:
# -----------------------------------
# 피처 추출(피처 엔지니어링)
# -----------------------------------
from sklearn.preprocessing import LabelEncoder

# 변수 PassengerId을 제거
train_x = train_x.drop(['PassengerId'], axis=1)
test_x = test_x.drop(['PassengerId'], axis=1)

# 변수 [ Name, Ticket, Cabin ]을 제거
train_x = train_x.drop(['Name', 'Ticket', 'Cabin'], axis=1)
test_x = test_x.drop(['Name', 'Ticket', 'Cabin'], axis=1)

# 범주형 변수에 label encoding 을 적용.
for c in ['Sex', 'Embarked']:
    # 학습 데이터를 기반으로 어떻게 변환 할지를 최적화 시킨다.
    le = LabelEncoder()
    le.fit(train_x[c].fillna('NA'))

    # 학습 데이터, 테스트 데이터를 변환
    train_x[c] = le.transform(train_x[c].fillna('NA'))
    test_x[c] = le.transform(test_x[c].fillna('NA'))


In [10]:
# -----------------------------------
# 모델 만들기
# -----------------------------------
from xgboost import XGBClassifier

# 모델 만들기 및 학습 데이터를 가지고 학습
model = XGBClassifier(n_estimators=20, random_state=71)
model.fit(train_x, train_y)

# 테스트 데이터의 예측치를 확률로 출력한다.
pred = model.predict_proba(test_x)[:, 1]

# 테스트 데이터의 예측치를 두개의 값(1,0)으로 변환
pred_label = np.where(pred > 0.5, 1, 0)

# 제출용 파일의 작성
submission = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': pred_label})
submission.to_csv('submission_first.csv', index=False)

In [11]:
# score：0.7799（이 수치는 사용자에 따라다를 수 있음.）

In [12]:
# -----------------------------------
# 모델 평가
# -----------------------------------
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import KFold

# 각 fold의 스코어를 저장을 위한 빈 리스트 작성
scores_accuracy = []
scores_logloss = []

# 교차 검증(Cross-validation)을 수행
# 01 학습 데이터를 4개로 분할
# 02 그중의 하나를 평가용 데이터 셋으로 한다.
# 03 이후 평가용 데이터 셋을 하나씩 옆으로 옮겨가며 검증을 수행
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_x):
    # 학습 데이터를 학습 데이터와 평가용 데이터 셋으로 나눈다.
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

    # 모델 학습을 수행
    model = XGBClassifier(n_estimators=20, random_state=71)
    model.fit(tr_x, tr_y)

    # 평가용 데이터의 예측 결과를 확률로 출력
    va_pred = model.predict_proba(va_x)[:, 1]

    # 평가용 데이터의 스코어를 계산
    logloss = log_loss(va_y, va_pred)
    accuracy = accuracy_score(va_y, va_pred > 0.5)

    # 각 fold의 스코어를 저장
    scores_logloss.append(logloss)
    scores_accuracy.append(accuracy)

# 각 fold의 스코어의 평균을 출력
logloss = np.mean(scores_logloss)
accuracy = np.mean(scores_accuracy)
print(f'logloss: {logloss:.4f}, accuracy: {accuracy:.4f}')

logloss: 0.4270, accuracy: 0.8148


In [13]:
# logloss: 0.4270, accuracy: 0.8148（결과값은 이 책과 다를 가능성이 있습니다.）

In [14]:
# -----------------------------------
# 모델 튜닝
# -----------------------------------
import itertools

# 튜닝 후보로 삼는 파라미터 값을 준비
param_space = {
    'max_depth': [3, 5, 7],
    'min_child_weight': [1.0, 2.0, 4.0]
}

# 탐색할 하이퍼 파라미터 값의 조합
param_combinations = itertools.product(param_space['max_depth'], param_space['min_child_weight'])

# 각 파라미터의 조합, 그에 대한 스코어를 보존하는 빈 리스트 
params = []
scores = []

# 각 파라미터 조합별로 교차 검증(Cross-validation)으로 평가를 수행
for max_depth, min_child_weight in param_combinations:

    score_folds = []
    # 교차 검증(Cross-validation)을 수행
    # 학습 데이터를 4개로 분할한 후, 
    # 그중 하나를 평가용 데이터로 하는 것을 데이터를 바꾸어 가면서 반복한다. 
    kf = KFold(n_splits=4, shuffle=True, random_state=123456)
    for tr_idx, va_idx in kf.split(train_x):
        # 학습 데이터를 학습 데이터와 평가용 데이터로 분할
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

        # 모델의 학습을 수행
        model = XGBClassifier(n_estimators=20, random_state=71,
                              max_depth=max_depth, min_child_weight=min_child_weight)
        model.fit(tr_x, tr_y)

        # 평가용 데이터의 스코어를 계산한 후, 저장
        va_pred = model.predict_proba(va_x)[:, 1]
        logloss = log_loss(va_y, va_pred)
        score_folds.append(logloss)

    # 각 fold의 스코어의 평균을 구한다.
    score_mean = np.mean(score_folds)

    # 파라미터를 조합, 그에 대한 스코어를 저장.
    params.append((max_depth, min_child_weight))
    scores.append(score_mean)

# 가장 스코어가 좋은 것을 베스트 파라미터로 한다.
# 最もスコアが良いものをベストなパラメータとする
best_idx = np.argsort(scores)[0]
best_param = params[best_idx]
print(f'max_depth: {best_param[0]}, min_child_weight: {best_param[1]}')
# max_depth=7, min_child_weight=2.0의 스코어가 가장 좋았음.

max_depth: 7, min_child_weight: 2.0


### 1.5.6 책에는 빠져 있음

In [15]:
# -----------------------------------
# 로지스틱 회귀용 feature 만들기
# -----------------------------------
from sklearn.preprocessing import OneHotEncoder

# 원 데이터를 복사한다.
train_x2 = train.drop(['Survived'], axis=1)
test_x2 = test.copy()

# 변수 PassengerId를 제거한다.
train_x2 = train_x2.drop(['PassengerId'], axis=1)
test_x2 = test_x2.drop(['PassengerId'], axis=1)

# 변수Name, Ticket, Cabin을 제거
train_x2 = train_x2.drop(['Name', 'Ticket', 'Cabin'], axis=1)
test_x2 = test_x2.drop(['Name', 'Ticket', 'Cabin'], axis=1)

In [16]:
train_x2.info()  # 추가 부분

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Pclass    891 non-null    int64  
 1   Sex       891 non-null    object 
 2   Age       714 non-null    float64
 3   SibSp     891 non-null    int64  
 4   Parch     891 non-null    int64  
 5   Fare      891 non-null    float64
 6   Embarked  889 non-null    object 
dtypes: float64(2), int64(3), object(2)
memory usage: 48.9+ KB


In [17]:
train_x2.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [18]:
# one-hot encoding을 수행 scikit - 0.22 버전 실행.
cat_cols = ['Embarked', 'Pclass', 'Sex']
ohe = OneHotEncoder(categories='auto', sparse=False)
ohe.fit(train_x2[cat_cols].fillna('NA'))

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=False)

In [19]:
# one-hot encoding의 더미 변수 컬럼명을 작성한다.
ohe_columns = []
for i, c in enumerate(cat_cols):
    ohe_columns += [f'{c}_{v}' for v in ohe.categories_[i]]
ohe_columns

['Embarked_C',
 'Embarked_NA',
 'Embarked_Q',
 'Embarked_S',
 'Pclass_1',
 'Pclass_2',
 'Pclass_3',
 'Sex_female',
 'Sex_male']

In [20]:
train_x2.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


In [21]:
# one-hot encodingd에 따른 변환 수행
ohe_train_x2 = pd.DataFrame(ohe.transform(train_x2[cat_cols].fillna('NA')), columns=ohe_columns)
ohe_test_x2 = pd.DataFrame(ohe.transform(test_x2[cat_cols].fillna('NA')), columns=ohe_columns)
ohe_train_x2

,Embarked_C,Embarked_NA,Embarked_Q,Embarked_S,Pclass_1,Pclass_2,Pclass_3,Sex_female,Sex_male
0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
1,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
3,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...
886,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
887,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
888,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
889,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [22]:
# one-hot encoding 끝난 변수를 제외하가
train_x2 = train_x2.drop(cat_cols, axis=1)
test_x2 = test_x2.drop(cat_cols, axis=1)

# one-hot encoding로 변환된 변수를 추가
train_x2 = pd.concat([train_x2, ohe_train_x2], axis=1)
test_x2 = pd.concat([test_x2, ohe_test_x2], axis=1)

# 수치 변수의 결측치를 학습 데이터 평균으로 메우기
num_cols = ['Age', 'SibSp', 'Parch', 'Fare']
for col in num_cols:
    train_x2[col].fillna(train_x2[col].mean(), inplace=True)
    test_x2[col].fillna(train_x2[col].mean(), inplace=True)

# 변수 Fare을 로그 변환하기
train_x2['Fare'] = np.log1p(train_x2['Fare'])
test_x2['Fare'] = np.log1p(test_x2['Fare'])

### 1.5.6 앙상블

In [23]:
# -----------------------------------
# 앙상블(ensemble)
# -----------------------------------
from sklearn.linear_model import LogisticRegression

# xgboost 모델
model_xgb = XGBClassifier(n_estimators=20, random_state=71)
model_xgb.fit(train_x, train_y)
pred_xgb = model_xgb.predict_proba(test_x)[:, 1]

# 로지스틱 회귀 모델
# xgboost 모델과는 다른 독립변수를 넣어야 하는 필요가 있으므로 별도로 train_x2, test_x2를 작성.
model_lr = LogisticRegression(solver='lbfgs', max_iter=300)
model_lr.fit(train_x2, train_y)
pred_lr = model_lr.predict_proba(test_x2)[:, 1]

# 예측 결과의 가중 평균을 취하다.
pred = pred_xgb * 0.8 + pred_lr * 0.2
pred_label = np.where(pred > 0.5, 1, 0)

In [24]:
pred_label

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,